In [1]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
import matplotlib.pyplot as plt
#import seaborn as sns
import datetime as dt
import os
import warnings
import matplotlib.dates as mpl_dates
from statsmodels.tsa.seasonal import seasonal_decompose
import operator
from mlforecast import MLForecast
from mlforecast.lag_transforms import Combine, RollingMean
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor
from statsforecast.models import AutoARIMA, AutoETS, Naive, RandomWalkWithDrift, SeasonalNaive, SeasonalWindowAverage, WindowAverage
import utilsforecast.losses as ufl
from utilsforecast.evaluation import evaluate



c:\Users\pc\Desktop\Proje_2\BF-final\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Loading data
future_values = pd.read_csv('future_values.csv',parse_dates = ['date']).rename(columns={'date':'ds','store_id':'unique_id'})
metadata = pd.read_csv('metadata.csv').rename(columns={'store_id':'unique_id'})
sales_data = pd.read_csv('sales_data.csv',parse_dates = ['date']).rename(columns={'date':'ds','store_id':'unique_id','sales':'y'})
sales_data.dtypes

C:\Users\pc\AppData\Local\Temp\ipykernel_19452\594441786.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_data = pd.read_csv('sales_data.csv',parse_dates = ['date']).rename(columns={'date':'ds','store_id':'unique_id','sales':'y'})


unique_id                 object
ds                datetime64[ns]
y                          int64
customers                  int64
open                       int64
promo                      int64
state_holiday             object
school_holiday             int64
dtype: object

In [ ]:
#Checking implicitly missing values 

# Grouping data by each time series
grouped = sales_data.groupby('unique_id')
# 5. Check min & max date per BU
print("\nDate range per unique_id:")
date_range = sales_data.groupby('unique_id')['ds'].agg(['min', 'max', 'count'])
print(date_range)
# Creating a summary dataframe for visualizing data completeness
summary = grouped.agg(
    count_observed=('ds', 'count'),
    start_date=('ds', 'min'),
    end_date=('ds', 'max')
).reset_index()

summary['expected_count'] = (
    (summary['end_date'].dt.to_period('M') - summary['start_date'].dt.to_period('M')).apply(lambda x: x.n) + 1
)
# Calculating expected number of weeks (one per week)
#summary['expected_count'] = ((summary['end_date'] - summary['start_date']) / pd.Timedelta(weeks=1)).round().astype(int) + 1

# Identifying which time series are irregular 
summary['is_irregular'] = summary['count_observed'] < summary['expected_count']

# show time series with implicitly missing values
filtered = summary[summary['is_irregular'] == True]
display(filtered)

#print(summary)



Date range per unique_id:
                 min        max  count
unique_id                             
store_1   2013-01-07 2015-07-19    924
store_10  2013-01-07 2015-07-19    924
store_100 2013-01-07 2015-07-19    924
store_101 2013-01-07 2015-07-19    924
store_102 2013-01-07 2015-07-19    924
...              ...        ...    ...
store_95  2013-01-07 2015-07-19    924
store_96  2013-01-07 2015-07-19    924
store_97  2013-01-07 2015-07-19    924
store_98  2013-01-07 2015-07-19    924
store_99  2013-01-07 2015-07-19    924

[676 rows x 3 columns]


,unique_id,count_observed,start_date,end_date,expected_count,is_irregular


In [5]:
# Grouping data by each time series
grouped = sales_data.groupby('unique_id')
summary = grouped.agg(
    count_observed=('ds', 'count'),
    start_date=('ds', 'min'),
    end_date=('ds', 'max')
).reset_index()
print(summary)

     unique_id  count_observed start_date   end_date
0      store_1             924 2013-01-07 2015-07-19
1     store_10             924 2013-01-07 2015-07-19
2    store_100             924 2013-01-07 2015-07-19
3    store_101             924 2013-01-07 2015-07-19
4    store_102             924 2013-01-07 2015-07-19
..         ...             ...        ...        ...
671   store_95             924 2013-01-07 2015-07-19
672   store_96             924 2013-01-07 2015-07-19
673   store_97             924 2013-01-07 2015-07-19
674   store_98             924 2013-01-07 2015-07-19
675   store_99             924 2013-01-07 2015-07-19

[676 rows x 4 columns]


In [6]:
# Grouping data by each time series
grouped = future_values.groupby('unique_id')
summary = grouped.agg(
    count_observed=('ds', 'count'),
    start_date=('ds', 'min'),
    end_date=('ds', 'max')
).reset_index()
print(summary)

     unique_id  count_observed start_date   end_date
0      store_1              60 2015-07-20 2015-09-17
1     store_10              60 2015-07-20 2015-09-17
2    store_100              60 2015-07-20 2015-09-17
3    store_101              60 2015-07-20 2015-09-17
4    store_102              60 2015-07-20 2015-09-17
..         ...             ...        ...        ...
671   store_95              60 2015-07-20 2015-09-17
672   store_96              60 2015-07-20 2015-09-17
673   store_97              60 2015-07-20 2015-09-17
674   store_98              60 2015-07-20 2015-09-17
675   store_99              60 2015-07-20 2015-09-17

[676 rows x 4 columns]


In [7]:
#Checking na 
future_values.isna ().sum ()
metadata.isna ().sum ()
sales_data.isna().sum()

unique_id         0
ds                0
y                 0
customers         0
open              0
promo             0
state_holiday     0
school_holiday    0
dtype: int64

In [9]:
sales_merged = pd.merge(sales_data, metadata, on='unique_id', how='left')
future_merged = pd.merge(future_values, metadata, on='unique_id', how='left')
sales_merged.head()

,unique_id,ds,y,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance
0,store_1,2015-07-19,0,0,0,0,0,0,c,a,1270.0
1,store_2,2015-07-19,0,0,0,0,0,0,a,a,14130.0
2,store_3,2015-07-19,0,0,0,0,0,0,a,c,24000.0
3,store_4,2015-07-19,0,0,0,0,0,0,a,a,7520.0
4,store_5,2015-07-19,0,0,0,0,0,0,a,c,2030.0


In [45]:
import pandas as pd

# Ensure datetime
sales_merged['ds'] = pd.to_datetime(sales_merged['ds'])

# Create a weekly bucket
sales_merged['week'] = sales_merged['ds'].dt.to_period('W-MON').dt.start_time

# Make sure state_holiday is string type
sales_merged['state_holiday'] = sales_merged['state_holiday'].astype(str)

# Count how many times each state_holiday type appears per week per store
holiday_counts = (
    sales_merged
    .groupby(['unique_id', 'week', 'state_holiday'])
    .size()
    .unstack(fill_value=0)  # turns into columns
    .reset_index()
    .rename_axis(None, axis=1)  # remove column name
)

# Optional: Rename columns for clarity
holiday_counts.columns = ['unique_id', 'week'] + [f'state_holiday_{col}' for col in holiday_counts.columns[2:]]

# Now aggregate your normal weekly data
weekly_data = sales_merged.groupby(['unique_id', 'week'], as_index=False).agg({
    'y': 'sum',
    'customers': 'sum',
    'promo': 'sum',
    'open': 'sum',
    'school_holiday': 'sum',
    'store_type': 'first',
    'assortment': 'first',
    'competition_distance': 'first'
})

# Merge the holiday counts in
weekly_data = weekly_data.merge(holiday_counts, on=['unique_id', 'week'], how='left')

# Fill in 0 where a holiday type didn’t occur that week
weekly_data.fillna(0, inplace=True)

weekly_data.head()

,unique_id,week,y,customers,promo,open,school_holiday,store_type,assortment,competition_distance,state_holiday_0,state_holiday_a,state_holiday_b,state_holiday_c
0,store_1,2013-01-01,7176,785,1,1,1,c,a,1270.0,1,0,0,0
1,store_1,2013-01-08,30493,3749,4,6,4,c,a,1270.0,7,0,0,0
2,store_1,2013-01-15,26655,3408,1,6,0,c,a,1270.0,7,0,0,0
3,store_1,2013-01-22,31732,3804,4,6,0,c,a,1270.0,7,0,0,0
4,store_1,2013-01-29,31670,3774,1,6,0,c,a,1270.0,7,0,0,0


In [ ]:
filtered_values = np.where((weekly_data['state_holiday_c']>0)) # & (weekly_data['state_holiday_b']> 0) & (weekly_data['state_holiday_c']>0))
print(filtered_values)
display(weekly_data.loc[filtered_values])

(array([   51,   103,   184, ..., 89745, 89826, 89878], shape=(1352,)),)


,unique_id,week,y,customers,promo,open,school_holiday,store_type,assortment,competition_distance,state_holiday_0,state_holiday_a,state_holiday_b,state_holiday_c
51,store_1,2013-12-24,22166,2634,0,4,7,c,a,1270.0,5,0,0,2
103,store_1,2014-12-23,24138,2762,0,4,7,c,a,1270.0,5,0,0,2
184,store_10,2013-12-24,28337,2893,0,4,5,d,c,4110.0,5,0,0,2
236,store_10,2014-12-23,31484,3189,0,4,5,d,c,4110.0,5,0,0,2
317,store_100,2013-12-24,25795,3109,0,4,5,c,a,4130.0,5,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89612,store_97,2014-12-23,23914,2554,0,4,5,d,a,980.0,5,0,0,2
89693,store_98,2013-12-24,20530,2250,0,4,5,a,a,1070.0,5,0,0,2
89745,store_98,2014-12-23,19415,2224,0,4,5,a,a,1070.0,5,0,0,2
89826,store_99,2013-12-24,29787,3156,0,4,5,a,a,2640.0,5,0,0,2


In [46]:
print(weekly_data['state_holiday_a']!= 0)

0        False
1        False
2        False
3        False
4        False
         ...  
89903    False
89904    False
89905    False
89906    False
89907    False
Name: state_holiday_a, Length: 89908, dtype: bool


In [39]:
# Create a new column "week"
sales_merged['week'] = sales_merged['ds'].dt.to_period('W-MON').dt.start_time

# Aggregate by store_id and week and take the sum
weekly_data = sales_merged.groupby(['unique_id', 'week'], as_index=False).agg({
    'y': 'sum',
    'customers': 'sum',
    'promo': 'sum',
    'open': 'sum',
    'school_holiday': 'sum',
    'store_type': 'first',
    'assortment': 'first',
    'competition_distance': 'first'
})
# 确保 state_holiday 是字符串类型
sales_merged['state_holiday'] = sales_merged['state_holiday'].astype(str)

# 创建 one-hot 编码列，例如：state_holiday_a, state_holiday_b, ...
state_holiday_dummies = pd.get_dummies(sales_merged['state_holiday'], prefix='state_holiday')

# 拼接到原始数据
sales_merged = pd.concat([sales_merged, state_holiday_dummies], axis=1)

# 按 store_id 和 week 对 one-hot 编码列求和
holiday_weekly = sales_merged.groupby(['unique_id', 'week'], as_index=False)[
    [col for col in state_holiday_dummies.columns]
].sum()

# 合并计数后的 holiday 列
weekly_data = pd.merge(weekly_data, holiday_weekly, on=['unique_id', 'week'], how='left')

weekly_data

,unique_id,week,y,customers,promo,open,school_holiday,store_type,assortment,competition_distance,...,state_holiday_b,state_holiday_b,state_holiday_b,state_holiday_b,state_holiday_c,state_holiday_c,state_holiday_c,state_holiday_c,state_holiday_c,state_holiday_c
0,store_1,2013-01-01,7176,785,1,1,1,c,a,1270.0,...,0,0,0,0,0,0,0,0,0,0
1,store_1,2013-01-08,30493,3749,4,6,4,c,a,1270.0,...,0,0,0,0,0,0,0,0,0,0
2,store_1,2013-01-15,26655,3408,1,6,0,c,a,1270.0,...,0,0,0,0,0,0,0,0,0,0
3,store_1,2013-01-22,31732,3804,4,6,0,c,a,1270.0,...,0,0,0,0,0,0,0,0,0,0
4,store_1,2013-01-29,31670,3774,1,6,0,c,a,1270.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89903,store_99,2015-06-16,51761,5071,4,6,0,a,a,2640.0,...,0,0,0,0,0,0,0,0,0,0
89904,store_99,2015-06-23,48696,4871,1,6,1,a,a,2640.0,...,0,0,0,0,0,0,0,0,0,0
89905,store_99,2015-06-30,55631,5181,4,6,5,a,a,2640.0,...,0,0,0,0,0,0,0,0,0,0
89906,store_99,2015-07-07,44007,4350,1,6,5,a,a,2640.0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
print(weekly_data['state_holiday_a']!= 0)

       state_holiday_a  state_holiday_a
0                False            False
1                False            False
2                False            False
3                False            False
4                False            False
...                ...              ...
89903            False            False
89904            False            False
89905            False            False
89906            False            False
89907            False            False

[89908 rows x 2 columns]


In [23]:
import numpy as np
from statsmodels.tsa.seasonal import STL

# Function to quantify seasonality strength and residual variability
def quantify_seasonality_and_residual(sales_data, seasonal_period=7, lo_frac=0.4):
    # Ensure datetime index
    ts = sales_data.set_index("ds").sort_index()

    # Run STL decomposition with safe parameters
    stl = STL(ts["y"], period=seasonal_period, robust=True, seasonal=7, trend=13)
    result = stl.fit()
    
    # Strength of seasonality: as per formula
    fs = max(0, 1 - np.var(result.resid) / np.var(result.resid + result.seasonal))

    # Residual variability
    rv = np.std(result.resid) / ts["y"].mean()

    return pd.Series({
        "strength_seasonality": round(fs, 2),
        "residual_variability": round(rv, 2)
    })
# Apply to each product
sales_data["ds"] = pd.to_datetime(sales_data["ds"])

seasonality_metrics = sales_data.groupby("unique_id").apply(
    lambda x: quantify_seasonality_and_residual(x, seasonal_period=12)
).reset_index()

print(seasonality_metrics)

     unique_id  strength_seasonality  residual_variability
0      store_1                  0.05                  0.48
1     store_10                  0.01                  0.50
2    store_100                  0.07                  0.47
3    store_101                  0.00                  0.59
4    store_102                  0.01                  0.53
..         ...                   ...                   ...
671   store_95                  0.02                  0.51
672   store_96                  0.00                  0.55
673   store_97                  0.01                  0.52
674   store_98                  0.00                  0.53
675   store_99                  0.00                  0.56

[676 rows x 3 columns]


C:\Users\pc\AppData\Local\Temp\ipykernel_19452\880849844.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  seasonality_metrics = sales_data.groupby("unique_id").apply(


In [30]:
import numpy as np
from statsmodels.tsa.seasonal import STL

# Function to quantify seasonality strength and residual variability
def quantify_seasonality_and_residual(weekly_data, seasonal_period=52, lo_frac=0.4): #52 bc yearly seasonality 
    # Ensure datetime index
    ts = weekly_data.set_index("ds").sort_index()

    # Run STL decomposition with safe parameters
    stl = STL(ts["y"], period=seasonal_period, robust=True, seasonal=7, trend=13)   ##look at how these are set 
    result = stl.fit()
    
    # Strength of seasonality: as per formula
    fs = max(0, 1 - np.var(result.resid) / np.var(result.resid + result.seasonal))

    # Residual variability
    rv = np.std(result.resid) / ts["y"].mean()

    return pd.Series({
        "strength_seasonality": round(fs, 2),
        "residual_variability": round(rv, 2)
    })
# Apply to each product
sales_data["ds"] = pd.to_datetime(sales_data["ds"])

seasonality_metrics = sales_data.groupby("unique_id").apply(
    lambda x: quantify_seasonality_and_residual(x, seasonal_period=12)
).reset_index()


print(seasonality_metrics)

     unique_id  strength_seasonality  residual_variability
0      store_1                  0.05                  0.48
1     store_10                  0.01                  0.50
2    store_100                  0.07                  0.47
3    store_101                  0.00                  0.59
4    store_102                  0.01                  0.53
..         ...                   ...                   ...
671   store_95                  0.02                  0.51
672   store_96                  0.00                  0.55
673   store_97                  0.01                  0.52
674   store_98                  0.00                  0.53
675   store_99                  0.00                  0.56

[676 rows x 3 columns]


C:\Users\pc\AppData\Local\Temp\ipykernel_19452\3041074722.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  seasonality_metrics = sales_data.groupby("unique_id").apply(


In [27]:
print(seasonality_metrics['strength_seasonality']>0.3)

0      False
1      False
2      False
3      False
4      False
       ...  
671    False
672    False
673    False
674    False
675    False
Name: strength_seasonality, Length: 676, dtype: bool


In [28]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL

def quantify_seasonality_and_residual(weekly_data, seasonal_period=52):
    ts = weekly_data.set_index("ds").sort_index()
    
    # STL with yearly seasonality for weekly data
    stl = STL(ts["y"], period=seasonal_period, robust=True)
    result = stl.fit()
    
    # Seasonality strength
    fs = max(0, 1 - np.var(result.resid) / np.var(result.resid + result.seasonal))
    
    # Residual variability
    rv = np.std(result.resid) / ts["y"].mean()
    
    return pd.Series({
        "strength_seasonality": round(fs, 2),
        "residual_variability": round(rv, 2)
    })


# Apply
seasonality_metrics = weekly_data.groupby("unique_id").apply(
    lambda x: quantify_seasonality_and_residual(x, seasonal_period=52)
).reset_index()

print(seasonality_metrics)


KeyError: "None of ['ds'] are in the columns"